# Experiment : Network growth

## Init the Ray client

In [1]:
# Imports
from clases.red_rddas_model import RedRddasModel
import ray
import time
import pandas as pd

# Ray Configurations
# runtime_env = {"working_dir": "/home/carlos/Documents/RESEARCH/SynEstRDDA", "pip": ["requests", "pendulum==2.1.2"]}
# ray.init(address='ray://172.17.163.253:10001', runtime_env=runtime_env , log_to_driver=False)
# ray.init(address='ray://172.17.163.244:10001', runtime_env=runtime_env , log_to_driver=False)
ray.init(log_to_driver=False, num_cpus=12)

RayContext(dashboard_url='', python_version='3.8.10', ray_version='1.13.0', ray_commit='e4ce38d001dbbe09cd21c497fedd03d692b2be3e', address_info={'node_ip_address': '172.17.163.244', 'raylet_ip_address': '172.17.163.244', 'redis_address': None, 'object_store_address': '/tmp/ray/session_2022-07-11_22-02-42_525027_1418180/sockets/plasma_store', 'raylet_socket_name': '/tmp/ray/session_2022-07-11_22-02-42_525027_1418180/sockets/raylet', 'webui_url': '', 'session_dir': '/tmp/ray/session_2022-07-11_22-02-42_525027_1418180', 'metrics_export_port': 53397, 'gcs_address': '172.17.163.244:52504', 'address': '172.17.163.244:52504', 'node_id': 'e2491922d1cdb8e1c7c534c750e24aa668ace2ed672ca4d03a1da7b4'})

### Generate the Samples and recollected the indicators

In [2]:
# Take the time of all Experiment
v_begin_exp = time.time()
# Experiment for RDD variation
n_experiments = 500
l_experiments = []
for cont_experiment in range(1,n_experiments+1):
    print("============================")
    print("Experiment:", cont_experiment)
    print("============================")

    # Variable Parameters
    n_rddas_min = 3
    n_rddas_max = 10

    # Fixed Parameters
    n_of_variables_rdda = 5
    n_of_signals_rdda = 2
    n_exit_variables = 2
    n_clauses_function = 2
    type_network = "ALEATORY"

    # List of Result for the Experiments
    l_res_sample = []

    v_n_network = 1
    for n_of_rdds in range(n_rddas_min, n_rddas_max + 1):
        print("Number of Network:", v_n_network)
        print("-------------------------------")

        # generate the RDDAs of the Network of RDDAs
        print("generating the Network of RDDAs ...")
        oRedRddasModel = RedRddasModel(n_of_rdds, n_of_variables_rdda, n_of_signals_rdda, n_exit_variables,
                                       n_clauses_function)

        # Generate the RDDs
        print("generating the rdds ...")
        oRedRddasModel.generate_rddas(type_network=type_network)

        # Calculate the Attractors by RDDA and by Signal
        v_begin_0 = time.time()
        result = RedRddasModel.find_attractors_rddas_ray.remote(oRedRddasModel)
        oRedRddasModel = ray.get(result)
        v_end_0 = time.time()
        v_time_0 = v_end_0 - v_begin_0

        # Calculate the Attractors by RDDA and by Signal
        v_begin_1 = time.time()
        result = RedRddasModel.calculation_compatible_pairs.remote(oRedRddasModel)
        oRedRddasModel = ray.get(result)
        v_end_1 = time.time()
        v_time_1 = v_end_1 - v_begin_1

        # # Calculate the Attractors by RDDA and by Signal with iterative Method
        # v_begin_1 = time.time()
        # # result = RedRddasModel.assembly_attractor_fields_iterative.remote(oRedRddasModel)
        # # oRedRddasModel = ray.get(result)
        # v_end_1 = time.time()
        # v_time_1 = v_end_1 - v_begin_1

        # Calculate the Attractors by RDDA and by Signal with optimized Method
        v_begin_2 = time.time()
        result = RedRddasModel.assembly_attractor_fields_optimized.remote(oRedRddasModel)
        oRedRddasModel = ray.get(result)
        v_end_2 = time.time()
        v_time_2 = v_end_2 - v_begin_2

        # Save the results for the experiment , numeric and time indicators
        res_dict = {
                    "n_network": v_n_network,
                    "n_rdds": n_of_rdds,
                    "n_rdda_attractors": len(oRedRddasModel.d_global_rdda_attractor.items()),
                    "t_find_attractors_method": v_time_0,
                    "n_pair_attractors": len(oRedRddasModel.list_attractors_pairs),
                    "t_comp_paris_method": v_time_1,
                    "n_attractor_fields": len(oRedRddasModel.attractor_fields),
                    "t_optimized_method": v_time_2
                    }
        l_res_sample.append(res_dict)
        v_n_network = v_n_network + 1

    # Add the sample data to pandas dataframe
    df = pd.DataFrame.from_dict(l_res_sample)
    l_experiments.append(df)
print("END EXPERIMENT")
# Take the time of the experiment
v_end_exp = time.time()
v_time_exp = v_end_exp - v_begin_exp
print("Time experiment (in seconds): ", v_time_exp )

Experiment: 1
Number of Network: 1
-------------------------------
generating the Network of RDDAs ...
generating the rdds ...
Number of Network: 2
-------------------------------
generating the Network of RDDAs ...
generating the rdds ...
Number of Network: 3
-------------------------------
generating the Network of RDDAs ...
generating the rdds ...
Number of Network: 4
-------------------------------
generating the Network of RDDAs ...
generating the rdds ...
Number of Network: 5
-------------------------------
generating the Network of RDDAs ...
generating the rdds ...
Number of Network: 6
-------------------------------
generating the Network of RDDAs ...
generating the rdds ...
Number of Network: 7
-------------------------------
generating the Network of RDDAs ...
generating the rdds ...
Number of Network: 8
-------------------------------
generating the Network of RDDAs ...
generating the rdds ...
Experiment: 2
Number of Network: 1
-------------------------------
generating the 

### Resume of the experiment

In [ ]:
# Total Time 359606.66107463837 segs  - 4,16 days
print("RESUME OF THE EXPERIMENT")
print("--------------------------------------------------------------------")
print("Name of the Experiment:", "Experiment 2 - Internal Variable Growth")
print("Variable Parameters : Number os RDDs")
print("Range of Number of RDDs:",n_rddas_min,"-", n_rddas_max)
print("Fixed parameters")
print("Number of variables:",n_of_variables_rdda)
print("Number of signals by RDD :", n_of_signals_rdda)
print("Number of output variables:", n_exit_variables)
print("Number of function clauses:", n_clauses_function)
print("Network Type:", type_network)
print("--------------------------------------------------------------------")
print("Time of Experiment (in seg)", v_time_exp)
print("Time of Experiment (in hours, minutes and seconds)", time.strftime("%H:%M:%S", time.gmtime(v_time_exp)))
print("--------------------------------------------------------------------")
pf_res = pd.concat(l_experiments, keys=range(1,n_experiments+1), names=["n_sample","n_aux"], ignore_index=False)
pf_res.reset_index(drop=True, inplace=True, level=1)
pf_res

## Save the experiment data in csv, using pandas Dataframe

In [3]:
pf_res = pd.concat(l_experiments, keys=range(1,n_experiments+1), names=["n_sample","n_aux"], ignore_index=False)
pf_res.reset_index(drop=True, inplace=True, level=1)
pf_res

,n_network,n_rdds,n_rdda_attractors,t_find_attractors_method,n_pair_attractors,t_comp_paris_method,n_attractor_fields,t_optimized_method
n_sample,,,,,,,,
1,1,3,29,1.428841,242,0.078654,6,0.037363
1,2,4,27,0.569466,170,0.042919,4,0.022636
1,3,5,33,1.312037,209,0.063791,9,0.064721
1,4,6,82,1.724923,904,0.212845,438,30.575644
1,5,7,68,1.613925,676,0.165403,172,3.318155
...,...,...,...,...,...,...,...,...
500,4,6,51,1.166447,399,0.095816,37,0.440454
500,5,7,66,1.587199,556,0.127697,55,3.661270
500,6,8,81,1.786939,726,0.165911,90,17.863223


In [4]:
path = "exp1_network_growth_data.csv"
pf_res.to_csv(path)
print("Experiment saved in:", path)

Experiment saved in: exp1_network_growth_data.csv
